# Extract eddy diffusivity data from Chapman & Sallée (2017) and interpolate them to our section coordinates

In [1]:
import pickle
import numpy as np
import xarray as xr
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d

## Load $K$ data from Chapman & Sallée 2017

Downloaded from https://datadryad.org/dataset/doi:10.5061/dryad.55bp8#citations

In [2]:
data = xr.open_dataset('data/ChapmanSallée_diffusivity/Diffusion_Coefficients_Filtered.nc')
lat = data.lat
lon = data.lon
depth = data.depth

#K0 = data.K_unsuppressed
K = data.K_suppressed_on_depth

In [3]:
# Select subset in Drake Passage
lat_sub = lat.sel(lat=slice(-62, -53))
lon_sub = lon.sel(lon=slice(300, 307))
depth_sub = depth.sel(depth=slice(0, 3000))
K_sub = K.sel(lat=slice(-62, -53), lon=slice(300, 307), depth=slice(0, 3000))

In [4]:
K_sub

<xarray.DataArray 'K_suppressed_on_depth' (depth: 40, lat: 37, lon: 29)> Size: 343kB
[42920 values with dtype=float64]
Coordinates:
  * lat      (lat) float64 296B -62.0 -61.75 -61.5 -61.25 ... -53.5 -53.25 -53.0
  * lon      (lon) float64 232B 300.0 300.2 300.5 300.8 ... 306.5 306.8 307.0
  * depth    (depth) float64 320B 25.0 75.0 125.0 ... 1.925e+03 1.975e+03

## Load coordinates from mooring data (depth + location on section)

In [5]:
# Moorings

ds_moorings_sr1b_depth = xr.open_dataset(f'data/variables_from_moorings_SR1b-lat_depth.nc')
depth_moorings = ds_moorings_sr1b_depth.depth
lat_moorings = ds_moorings_sr1b_depth.lat

with open('data/pp_lat_lon.pkl', 'rb') as f:
    pp_lat_lon = pickle.load(f)

lon_moorings = pp_lat_lon(lat_moorings) + 360 # add 360 shift for compatibility with grid

## Interpolate $K$ to mooring coordinates

In [6]:
# interpolate in x,y
K_moorings = np.zeros((len(depth), len(lat_moorings)))
points = np.array([(lat, lon) for lat,lon in zip(lat_moorings,lon_moorings)])
for i in range(len(depth)):
    f_K = RegularGridInterpolator((lat_sub, lon_sub), K_sub.values[i,:,:], bounds_error=False,fill_value=np.nan)
    K_moorings[i,:] = f_K(points)

# interpolate in z
K_moorings_interp = np.zeros((len(depth_moorings), len(lat_moorings)))
for i in range(len(lat_moorings)):
    f_K = interp1d(depth, K_moorings[:,i], bounds_error=False, fill_value=np.nan)
    K_moorings_interp[:,i] = f_K(depth_moorings)

In [7]:
ds = xr.Dataset({"K": (["depth", "lat"], K_moorings_interp)},
                coords={"depth": depth_moorings, "lat": lat_moorings})
ds.to_netcdf('data/ChapmanSallée_diffusivity/eddy_diff_ChapmanSallée.nc')